# Explore here

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import *
from imblearn.metrics import specificity_score
import pandas as pd

In [2]:
# VER BASE DE DATOS:

df_diabetes = pd.read_csv('../data/raw/diabetes.csv')
df_diabetes

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [3]:
X_train = df_diabetes.drop('Outcome', axis=1).values
y_train = df_diabetes['Outcome'].values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [5]:
# BOOSTING PERSONALIZADO:

class CustomBoosting:

  def __init__(self, n_estimators=100, learning_rate=0.01, max_depth=10, min_samples_leaf=10, max_features=1.0, random_state=42):
    # Inicializa el modelo con hiperparámetros personalizables
    self.n_estimators = n_estimators  # Número de estimadores en el ensamble
    self.learning_rate = learning_rate  # Tasa de aprendizaje
    self.max_depth = max_depth  # Profundidad máxima de los árboles de decisión
    self.min_samples_leaf = min_samples_leaf  # Número mínimo de muestras por hoja
    self.max_features = max_features  # Proporción máxima de características a considerar en cada división
    self.random_state = random_state  # Semilla aleatoria para reproducibilidad
    self.estimadores = [] # Almacena los estimadores del ensamble

  def muestreo(self, X, y, pesos):
      # Realiza un muestreo ponderado de las instancias de entrenamiento
      np.random.seed(self.random_state)
      indices = np.random.choice(len(X), len(X), p=pesos)  # Selecciona posiciones del conjunto de datos con reemplazo
      return X[indices], y[indices]

  def get_error(self, y, y_pred, pesos):
      # Calcula el error ponderado, indicando cuántas instancias se clasificaron incorrectamente en relación a sus pesos
      instancias_erroneas = (y_pred != y)  # Identifica las instancias mal clasificadas (True/False)
      pesos_instancias_erroneas = pesos[instancias_erroneas]  # Obtiene los pesos de las instancias erróneas
      suma_peso_error = np.sum(pesos_instancias_erroneas)  # Suma los pesos de las instancias erróneas
      error_ponderado = suma_peso_error / np.sum(pesos)  # Proporciona un valor entre 0 y 1
      return instancias_erroneas, error_ponderado

  def get_estimator_importance(self, error_ponderado):
      # Calcula la importancia del estimador en el ensamble
      ratio_acierto_error = (1 - error_ponderado) / error_ponderado  # Mide el grado de acierto del estimador (mayor valor indica más aciertos que fallos)
      ratio_log = np.log(ratio_acierto_error)  # Aumenta el impacto de los estimadores que funcionan mejor y reduce el de los que funcionan peor
      # Ajustando su influencia en función de su rendimiento y asegurando un equilibrio entre el aprendizaje eficaz y la prevención del overfitting
      estimador_imp = self.learning_rate * ratio_log  # Limita el impacto de cada estimador en la predicción final (para evitar el overfitting)
      return estimador_imp

  def update_weights(self, estimador_imp, instancias_erroneas, pesos):
      # Actualiza los pesos de las instancias para dar más importancia a las instancias mal clasificadas
      # incrementar de forma más específica los pesos de las instancias que fueron mal clasificadas,
      # mientras que disminuye los de las que fueron correctamente clasificadas, ajustando así de manera más efectiva la atención del modelo hacia las instancias que más necesita aprender.
      actualizacion_pesos = np.where(instancias_erroneas, np.exp(estimador_imp), np.exp(-estimador_imp))
      pesos = pesos * actualizacion_pesos  # Aplica la actualización a los pesos
      pesos = pesos / np.sum(pesos)  # Normaliza los pesos para que sumen 1, creando una distribución de probabilidad válida
      return pesos

  def fit(self, X, y):
    # Inicializar los pesos de las instancias de forma que todas tengan la misma probabilidad de salir
    pesos = np.ones(len(X)) / len(X)
    # Para cada árbol
    for _ in range(self.n_estimators):
      # Realizamos el muestreo de las instancias para entrenar el árbol
      X_sampled, y_sampled = self.muestreo(X, y, pesos)
      # Realizamos el ajuste del árbol a ese dataset muestreado
      estimador = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_leaf=self.min_samples_leaf, max_features=self.max_features).fit(X_sampled, y_sampled)
      # Aplico el árbol entrenado sobre mi X
      y_pred = estimador.predict(X)
      # Obtenemos el error cometido
      instancias_erroneas, error_ponderado = self.get_error(y, y_pred, pesos)
      # Obtener la importancia de ese árbol
      estimador_imp = self.get_estimator_importance(error_ponderado)
      # Almacena el estimador y su importancia asociada
      self.estimadores.append((estimador, estimador_imp))
      # Actualizar los pesos
      pesos = self.update_weights(estimador_imp, instancias_erroneas, pesos)

  def predict(self, X):
    # Inicializa un vector de predicciones vacío con el mismo número de instancias a predecir
    final_predictions = np.zeros(X.shape[0])
    # Para cada estimador en el ensamble
    for estimador, estimador_imp in self.estimadores:
        # Realiza predicciones con las instancias
        predictions = estimador.predict(X)
        # Pondera las predicciones con la importancia del estimador
        preds_ponderadas = estimador_imp * predictions
        # Acumula las predicciones ponderadas de cada estimador
        final_predictions = final_predictions + preds_ponderadas
    mean_val = np.mean(final_predictions)
    yhats = np.where(final_predictions<mean_val,0,1)
    return yhats

In [6]:
# ENTRENAMIENTO DEL MODELO DE BOOSTING:
clf = CustomBoosting(n_estimators=10, learning_rate=0.01, max_depth=7, min_samples_leaf=7, max_features=X_train.shape[1]//2)
clf.fit(X_train, y_train)

In [7]:
# FUNCIÓN DE MÉTRICAS:
def get_metrics(y_train, y_test, y_pred_train, y_pred_test):
    # Calcular métricas para el conjunto de entrenamiento
    train_accuracy = accuracy_score(y_train, y_pred_train)
    train_f1 = f1_score(y_train, y_pred_train)
    train_auc = roc_auc_score(y_train, y_pred_train)
    train_precision = precision_score(y_train, y_pred_train)
    train_recall = recall_score(y_train, y_pred_train)
    train_specificity = specificity_score(y_train, y_pred_train)

    # Calcular métricas para el conjunto de prueba
    test_accuracy = accuracy_score(y_test, y_pred_test)
    test_f1 = f1_score(y_test, y_pred_test)
    test_auc = roc_auc_score(y_test, y_pred_test)
    test_precision = precision_score(y_test, y_pred_test)
    test_recall = recall_score(y_test, y_pred_test)
    test_specificity = specificity_score(y_test, y_pred_test)

    # Calcular la diferencia entre métricas de entrenamiento y prueba
    diff_accuracy = train_accuracy - test_accuracy
    diff_f1 = train_f1 - test_f1
    diff_auc = train_auc - test_auc
    diff_precision = train_precision - test_precision
    diff_recall = train_recall - test_recall
    diff_specificity = train_specificity - test_specificity

    # Crear un DataFrame con los resultados
    metrics_df = pd.DataFrame([[train_accuracy, train_f1, train_auc, train_precision, train_recall, train_specificity],[test_accuracy, test_f1, test_auc, test_precision, test_recall, test_specificity],[diff_accuracy, diff_f1, diff_auc, diff_precision, diff_recall, diff_specificity]],
                              columns = ['Accuracy', 'F1', 'AUC', 'Precision', 'Recall', 'Specificity'],
                              index = ['Train','Test', 'Diferencia'])

    return metrics_df

In [8]:
# PREDICIONES DEL CONJUNTO DE ENTRENAMIENTO Y PRUEBA:
test_pred = clf.predict(X_test)
train_pred = clf.predict(X_train)

# EVALUACIÓN DEL MODELO:
get_metrics(y_train, y_test, train_pred, test_pred)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.840391,0.787879,0.843689,0.730924,0.854460,0.832918
Test,0.779221,0.716667,0.779798,0.661538,0.781818,0.777778
Diferencia,0.061170,0.071212,0.063891,0.069385,0.072642,0.055140
